<h1 style="text-align : center"> FIXED POINT PATCH CODE REVIEW </h1>

# Table of Contents
* [Introduction](#Introduction)
* [Gossary](#Gossary)
* [Review](#Review)
	* [/include/clang/Basic/TokenKinds.def](#/include/clang/Basic/TokenKinds.def)
	* [/include/clang/Sema/DeclSpec.h](#/include/clang/Sema/DeclSpec.h)
	* [/lib/Parse/ParseDecl.cpp](#/lib/Parse/ParseDecl.cpp)
	* [/lib/Sema/SemaType.cpp](#/lib/Sema/SemaType.cpp)
	* [/include/clang/AST/ASTContext.h](#/include/clang/AST/ASTContext.h)
	* [/include/clang/AST/Expr.cpp](#/include/clang/AST/Expr.cpp)
	* [/lib/Sema/SemaExpr.cpp](#/lib/Sema/SemaExpr.cpp)


# Introduction

1. This patch includes the diffs for the lexer, parser, sema, and conversion to bitcode (clang/lib/CodeGen).
2. The diffs are against the **LLVM 3.7**.
3. The patch **doesn’t support unsigned nor unsaturated _Fract or _Accum types**

# Gossary

- [**Type**](http://clang.llvm.org/doxygen/classclang_1_1Type.html) : The base class of the type hierarchy.
- [**QualType**](http://clang.llvm.org/doxygen/classclang_1_1QualType.html) : Wrapper class that has two pointers as attributes: one points to the Type and the other points to the Qualifier (Both of these classes are defined in Type.h). This greatly reduces the number of nodes we need to allocate for types. For the sake of semplicity we will consider QualType equals to CanQualType

<img  src="FQWHQFGMBF9OHV9MVBMGARA9R4UYDU95.png"/>

- [**TST**](http://clang.llvm.org/doxygen/namespaceclang.html#a033691a5f00979c1a22d8aa114d07e13) : The Type Specifier Type (TST) is used to just specifies the type using an enum.
- [**DeclSpec**](http://clang.llvm.org/doxygen/classclang_1_1DeclSpec.html) : A Declaration Specifier encompasses storage-class-specifiers, type-specifiers, type-qualifiers, and function-specifiers. It basically describes the content of a declaration.
- [**Sema**](http://clang.llvm.org/doxygen/classclang_1_1Sema.html): Implements the semantic analysis and AST building for C.

# Review

## /include/clang/Basic/TokenKinds.def

In this file we added the definition of the token that the lexer will find inside the source code:
```C++
KEYWORD(__int96    , KEYALL)
KEYWORD(_Sat         , KEYALL)   // As specified by the ISO/IEC TR 18037
KEYWORD(_Fract      , KEYALL)   // As specified by the ISO/IEC TR 18037
KEYWORD(_Accum   , KEYALL)   // As specified by the ISO/IEC TR 18037
```

## /include/clang/Sema/DeclSpec.h

Added the new Type Specifier Type for the new type. 
```C++
static const TST TST_fract = clang::TST_fract;
static const TST TST_accum = clang::TST_accum;
static const TST TST_int96 = clang::TST_int96;
```

Addded the function **setSaturated** in order to memorize this information while parsing the declaration
The function setSaturated is used inside **Parser::ParseDeclarationSpecifiers** when meet a kw__Sat.

Saturated is a global variable inside this file that indicates if a declaration is a Sat or not
```C++
void setSaturated(SourceLocation l) { Saturated = true; SaturatedLoc = l; }
bool isSaturated() const { return Saturated; }
```

## /lib/Parse/ParseDecl.cpp

in the function **Parser::ParseDeclarationSpecifiers**  ( row 2618 ) added the cases in order to handle the new type. For example in the case of the kw__Sat we are going to mark as Saturated the DS, using the function previously defined setSaturated().
```C++
ParseGNUAttributes(DS.getAttributes(), nullptr, LateAttrs);
continue;
  
+ //<Synopsys> FixedPoint
+     case tok::kw__Sat:
+       if (DS.isSaturated()) Diag(Tok, diag::err_sat_specified);
+       DS.setSaturated(Tok.getLocation());
+       break;
+     case tok::kw__Fract:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_fract, Loc, PrevSpec,
+                                                  DiagID, Policy);
+       break;
+     case tok::kw__Accum:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_accum, Loc, PrevSpec,
+                                                              DiagID, Policy);
+       break;
+     case tok::kw___int96:
+       isInvalid = DS.SetTypeSpecType(DeclSpec::TST_int96, Loc, PrevSpec,
+                                                                DiagID, Policy);
+       break;
+ //</Synopsys> FixedPoint
+ 
// Microsoft declspec support.
case tok::kw___declspec:
ParseMicrosoftDeclSpecs(DS.getAttributes());
```

Added in the function **Parser::isKnownToBeTypeSpecifier** the cases in order to understand if we have a known type specifier.
```C++
    case tok::kw__Decimal64:
    case tok::kw__Decimal128:
    case tok::kw___vector:
+ //<Synopsys> FixedPoint
+   case tok::kw__Sat:
+   case tok::kw__Fract:
+   case tok::kw__Accum:
+   case tok::kw___int96:
+ //</Synopsys> FixedPoint
```

## /lib/Sema/SemaType.cpp

Here too, added the cases for handling the new TST and fill correctly the Declaration Specifier ( DS ).

## /include/clang/AST/ASTContext.h

added the new CanQualType (see QualType) in order to handle the new fixed point type and the respective getter.
```C++
+ //<Synopsys> FixedPoint
+   CanQualType SatSignedShortFractTy;
+   CanQualType SatSignedFractTy;
+   CanQualType SatSignedLongFractTy;
+   CanQualType SatSignedLongLongFractTy;
+   CanQualType SatSignedShortAccumTy;
+   CanQualType SatSignedAccumTy;
+   CanQualType SatSignedLongAccumTy;
+   CanQualType SatSignedLongLongAccumTy;
+   CanQualType Int96Ty, UnsignedInt96Ty;
+ //</Synopsys> FixedPoint
+ 
```
```C++
+   QualType getInt96Ty() const { return Int96Ty; }
+   QualType getUnsignedInt96Ty() const { return UnsignedInt96Ty; }
+   QualType getSatSignedShortFractTy() const { return SatSignedShortFractTy; }
+   QualType getSatSignedFractTy() const { return SatSignedFractTy; }
+   QualType getSatSignedLongFractTy() const { return SatSignedLongFractTy; }
+   QualType getSatSignedLongLongFractTy() const { return SatSignedLongLongFractTy; }
+   QualType getSatSignedShortAccumTy() const { return SatSignedShortAccumTy; }
+   QualType getSatSignedAccumTy() const { return SatSignedAccumTy; }
+   QualType getSatSignedLongAccumTy() const { return SatSignedLongAccumTy; }
+   QualType getSatSignedLongLongAccumTy() const { return SatSignedLongLongAccumTy; }
+ 
+   QualType getSignedFractType(unsigned SizeInBits);
+   QualType getFixedPointType(const llvm::fltSemantics&) const;
+   QualType getMaximalAccumType(QualType) const;
+   QualType getAccumTypeForFractType(QualType) const;
```
As we can see here the getters return the CanQualType defined before as a QualType.

## /include/clang/AST/Expr.cpp

Implementation of the function **commuteFixedPoint()**,
Basically if the type of the operand involved in the expression is Fract or Accum, if we match one of the operation in the switch (only the commutative operations) then we are going to swap the SubExprs[0] and SubExprs[1] (SubExprs is an array of Stmt). This swap can be useful in order to simplify some cases when we have to deal with cast ecc.. because we know for sure that if a fixed point type is involved in the expression this will be for sure the left operand (maybe?). 

```C++
+ void BinaryOperator::commuteFixedPoint() {
+   if (getType()->isFractOrAccumType() && getLHS()->getType()->isIntegerType()) {
+     switch (getOpcode()) {
+     case BO_Mul:
+     case BO_Add:
+     case BO_EQ:
+     case BO_NE:
+     case BO_And:
+     case BO_Xor:
+     case BO_Or:
+       std::swap(SubExprs[0], SubExprs[1]);
+       break;
+     default:
+       break;
+     }
+   }
+ }
```
In this class we update also the getter and setter of the semantics in order to include the new types.
```C++
// getSemantics()
+   case SignedShortFract:
+     return llvm::APFloat::SignedShortFract;
+   case SignedFract:
+     return llvm::APFloat::SignedFract;
+   case SignedLongFract:
+     return llvm::APFloat::SignedLongFract;
+   case SignedLongLongFract:
+     return llvm::APFloat::SignedLongLongFract;
+   case SignedShortAccum:
+     return llvm::APFloat::SignedShortAccum;
+   case SignedAccum:
+     return llvm::APFloat::SignedAccum;
+   case SignedLongAccum:
+     return llvm::APFloat::SignedLongAccum;
+   case SignedLongLongAccum:
+     return llvm::APFloat::SignedLongLongAccum;
```
```C++
// setSemantics()
+   else if (&Sem == &llvm::APFloat::SignedShortFract)
+     FloatingLiteralBits.Semantics = SignedShortFract;
+   else if (&Sem == &llvm::APFloat::SignedFract)
+     FloatingLiteralBits.Semantics = SignedFract;
+   else if (&Sem == &llvm::APFloat::SignedLongFract)
+     FloatingLiteralBits.Semantics = SignedLongFract;
+   else if (&Sem == &llvm::APFloat::SignedLongLongFract)
+     FloatingLiteralBits.Semantics = SignedLongLongFract;
+   else if (&Sem == &llvm::APFloat::SignedShortAccum)
+     FloatingLiteralBits.Semantics = SignedShortAccum;
+   else if (&Sem == &llvm::APFloat::SignedAccum)
+     FloatingLiteralBits.Semantics = SignedAccum;
+   else if (&Sem == &llvm::APFloat::SignedLongAccum)
+     FloatingLiteralBits.Semantics = SignedLongAccum;
+   else if (&Sem == &llvm::APFloat::SignedLongLongAccum)
+     FloatingLiteralBits.Semantics = SignedLongLongAccum;
```

## /lib/Sema/SemaExpr.cpp

This file implements semantic analysis for expressions.



Added inside the function **handleFloatConversion()**

Basically it checks the type of the two operands and insert inside the builded expression's tree, the operation in order to perform the case to the more precise type.

```C++
! //<Synopsys> FixedPoint
!   if (LHSType->isFractOrAccumType() && RHSType->isIntegerType()) {
!     QualType T = S.Context.getMaximalAccumType(LHSType);
!     RHS = S.ImpCastExprToType(RHS.get(), T, CK_FloatingCast);
!     RHSType = T;
!   }
!   else if (RHSType->isFractOrAccumType() && LHSType->isIntegerType()) {
!     QualType T = S.Context.getMaximalAccumType(RHSType);
!     LHS = S.ImpCastExprToType(LHS.get(), T, CK_FloatingCast);
!     LHSType = T;
!   }
! //</Synopsys> FixedPoint
! 
!   bool LHSFloat = LHSType->isRealOrFixed();
!   bool RHSFloat = RHSType->isRealOrFixed();
  
    // If we have two real floating types, convert the smaller operand
    // to the bigger result.
    if (LHSFloat && RHSFloat) {
      int order = S.Context.getFloatingTypeOrder(LHSType, RHSType);
+     if (order == 0) // Seen when error nodes in tree  -- SYNOPSYS
+       return LHSType;
      if (order > 0) {
        RHS = S.ImpCastExprToType(RHS.get(), LHSType, CK_FloatingCast);
        return LHSType;
```

HandleFloatConversion basically introduces the Expression in order to perform the casts
```C++
+ bool Type::isRealOrFixed() const {
+   return isRealFloatingType() || isFractOrAccumType();
+ }
```

- **/lib/AST/Type.cpp**
    - **isFractOrAccumType()** : It checks if the type identifier is included in the bounds reserved to the fixed poind types (specified in the class "BuiltinType")
    - **getMaximalAccumType(QualType qt)** :

# Questions
- In the file Expr.cpp what does it mean " Allow an intrinsic function call to be used for a fixed point static initializer"? what is the point of this code?

```C++
+   case CallExprClass:{
+     const CallExpr* call = cast<CallExpr>(this);
+     if (unsigned id = call->getBuiltinCallee()) {
+       if (call->getNumArgs() == 1) {
+ 	switch (id) {
+ 	case clang::ARC::BI__int_to_q7:
+ 	case clang::ARC::BI__int_to_q15:
+ 	case clang::ARC::BI__int_to_q31:
+ 	case clang::ARC::BI__int_to_a24:
+ 	case clang::ARC::BI__int_to_a40:
+ 	case clang::ARC::BI__int_to_a72:
+ 	case clang::ARC::BI__q7_to_int:
+ 	case clang::ARC::BI__q15_to_int:
+ 	case clang::ARC::BI__q31_to_int:
+ 	case clang::ARC::BI__a24_to_int:
+ 	case clang::ARC::BI__a40_to_int:
+ 	case clang::ARC::BI__a72_to_int:
+ 	  return call->getArg(0)->isConstantInitializer(Ctx, false, Culprit);
+ 	default: break;
+ 	}
+       }
+     }
+   } break;
```